# Mass evolution plot

In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import dill

import dustpy
import dustpy.constants as c

if '..' not in sys.path:
    sys.path.append('..')


import helper
import plot_helper

helper.set_paletton_colors([3,1,2,0])

Read the data sets

In [ ]:
#data_dirs = list((Path('~').expanduser() / 'DATA/araa_data/lowres_highvf').glob('data_*'))
data_dirs = list((Path('~').expanduser() / 'DATA/araa_data/lowerres_dipsy6').glob('data_*'))

data_sets = {}
for i, data_dir in enumerate(data_dirs[::-1]):

    writer = dustpy.hdf5writer()
    writer.datadir = str(data_dir)
    
    data = writer.read.output(0)
    data_sets[data_dir.name] = data
   
    data.time = writer.read.sequence('t')
    data.sigg = writer.read.sequence('gas.Sigma')
    data.sigd = writer.read.sequence('dust.Sigma')
    
    data.Mg = (data.grid.A * data.sigg).sum(-1)
    data.Md = (data.grid.A * data.sigd.sum(-1)).sum(-1)

    data.data_dir = data_dir

    # rc is not contained in the snapshots, so we need the frames:
    with open(data_dir / 'frame.dmp', 'rb') as fid:
        frame = dill.load(fid)
        data.rc = frame.ini.gas.SigmaRc / c.au
        del frame

In [ ]:
fig, ax = plt.subplots(figsize=(6.4, 4))
ax.loglog()
xlim = [1e3, 3e6]
ylim = [5e-3, 2]

ax.set_xlim(xlim)
ax.set_ylim(ylim)

# get conversion functions

axis_to_data = ax.transAxes + ax.transData.inverted()
data_to_axis = axis_to_data.inverted()#

# plot the mass tracks

for i, (data_dir, data) in enumerate(data_sets.items()):
    

    ######### draw the gas and dust mass evolution
    
    l1, = ax.loglog(data.time / c.year, data.Mg / data.Mg[0], c=f'C{i+2}', ls='--')
    l2, = ax.loglog(data.time / c.year, data.Md / data.Md[0], c=f'C{i+2}', ls='-')

    ax.loglog([], c=l1.get_color(), label='with' + 'out' * ('no' in str(data_dir)) +' pressure bump')

    
    if ('_bump' in str(data_dir)):
        ######### Compute dust mass outside planet and display it #########
        i_p = data.grid.r.searchsorted(data.planets.jupiter.a[0])
        m_out = (data.dust.Sigma.sum(-1) * data.grid.A)[i_p:].sum()
        _ = ax.axhline(m_out / data.Md[0], c='0.35', ls='--', zorder=1)
        ax.text(axis_to_data.transform([0.2, 0])[0], m_out / data.Md[0],
                r'dust mass at $r>r_\mathrm{gap}$',
                ha='center', va='bottom', fontsize=8, c=_.get_color())
    else:

        ############# compute and show growth time scale ###############
        
        Ng = 4
        Nr = 2
        tgrow = np.interp(Nr * data.rc * c.au, data.grid.r, Ng / (data.grid.OmegaK * data.dust.eps))
        ax.axvline(tgrow / c.year, c='k', ls='-.')
        ax.text(data_to_axis.transform([tgrow / c.year, 0])[0], 0.5,
        fr'${Ng} \times t_\mathrm{{grow}} \mathsf{{~at~}} {Nr} \times r_c$',
        va='center', ha='right', rotation=90, transform=ax.transAxes)

        ############ viscous time scale ############
        tnu = np.interp(data.rc * c.au, data.grid.r, data.grid.r**2 / data.gas.nu)
        if tnu / c.year < ax.get_xlim()[0]:
            ax.axvline(tnu / c.year, c='k', ls='-.')
            ax.text(data_to_axis.transform([tnu / c.year, 0])[0], 0.5,
                fr'$t_\mathrm{{\nu}} \mathsf{{~at~}} r_c$',
                va='center', ha='right', rotation=90, transform=ax.transAxes)
        
        ######## analytical curve ########
        
        Md0 = data.Md[0]
        eps0 = Md0 / data.Mg[0]

        time2 = np.maximum(0, data.time - tgrow)
        
        # compute omega and viscous time scale at the appropriate radii
        omega = np.interp(Nr * data.rc, data.grid.r / c.au, data.grid.OmegaK) 
        md_ana = Md0 / (1 + eps0 * time2 * omega) / np.sqrt(time2 / tnu + 1)
        #ax.loglog(data.time / c.year, md_ana / Md0)

        

ax.set_xlabel('time [yr]')
ax.set_ylabel('mass / initial mass')

ax.loglog([], c='k', ls='--', label='gas')
ax.loglog([], c='k', ls='-', label='dust')

########## show the t^-1 line #############
_t = np.array([1e3, 3e6])
for slope in [-1,-1.5]:
    f = lambda t: 0.1 / abs(slope) * (t / 2e5)**slope
    ax.loglog(_t, f(_t), c='k', ls=':')
    angle = np.rad2deg(np.arctan(slope)) #+ np.rad2deg(np.arctan(np.log10(ylim[1]/ylim[0]) / np.log10(xlim[1]/xlim[0])))
    
    ax.text(3e5, f(3e5), rf'$\propto t^{{{slope}}}$',
        ha='center', va='top', #transform=ax.transAxes,
        rotation=angle)

ax.legend().get_frame().set_alpha(0)

#### background coloring ####

fill = [xlim[0], tgrow / c.year]
y0 = 1.3

# first fill is normal
ax.fill_between(fill, ylim[0] * np.ones(2), y2=ylim[1] * np.ones(2),
                color=plt.get_cmap('Blues')(1.0),
                alpha=0.5, ec='none', zorder=0)

# add another axes to overlay an image
_x0, _y0 = data_to_axis.transform([tgrow / c.year, ylim[0]])
_x1, _y1 = data_to_axis.transform([xlim[1], ylim[1]])
axi = ax.inset_axes([_x0, _y0, _x1 - _x0, _y1 - _y0])
plot_helper.plot_blue_green_transition(axi)
axi.zorder=0

ax.set_xlim(xlim)
ax.set_ylim(ylim)    

# text for the 3 epochs

opts = dict(ha='center', weight='bold', fontsize=8)
ax.text(10.**np.mean(np.log10(fill)), y0, 'initial growth', **opts)
ax.text(1.5e5, y0, 'frag. limited', **opts)
ax.text(1.3e6, y0, 'drift limited', **opts)

ax.set_facecolor('none')
helper.apply_araa_style(ax)
output_file = helper.output_dir / 'BirnstielFig8.pdf'
fig.savefig(output_file)